# Notebook Downloading Mars Seismic Data, Splitting Windows
This notebook downloads mseed data from IRIS and splits it into 12 second windows sliding by 3 seconds

In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

import csv, json, os, sys
import requests
from time import sleep

import obspy
from obspy import read

from obspy.core.event import read_events
from obspy import read_inventory
from datetime import datetime, timedelta
from obspy.core import UTCDateTime


DATA_PATH = os.path.expanduser('~/dev/seisml/experiments/mars_insight_clustering/.data/')

## Download and Sample
the following:
* net=XB
* sta=ELYSE
* cha=BHU,BHV,BHW
* loc=02

In [3]:
def download_data_availability(data_path=DATA_PATH):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass
    
    payload = {
        'option': 'com_ajax',
        'data': 'ELYSE',
        'format': 'json',
        'module': 'seis_data_available'
    }
    r = requests.post('https://www.seis-insight.eu/en/science/seis-data/seis-data-availability', payload)
    with open(os.path.join(data_path, 'data_availability.json'), 'wb') as f:
        f.write(r.content)

In [4]:
download_data_availability()

In [5]:
with open(os.path.join(DATA_PATH, 'data_availability.json'), 'r') as f:
    raw_ava = json.load(f)

In [6]:
ava = []
for t in raw_ava['data']:
    if t['network'] == 'XB' and t['location'] == '02' and t['channel'] == 'BHU':
        ava.append(t)

In [7]:
ava[0]

{'network': 'XB',
 'station': 'ELYSE',
 'location': '02',
 'channel': 'BHU',
 'startTime': '2019-02-12T02:43:01.313000',
 'endTime': '2019-02-12T04:14:37.259000',
 'sampleRate': '20',
 'quality': 'R'}

example data req: `http://ws.ipgp.fr/fdsnws/dataselect/1/query?network=XB&amp;station=ELYSE&amp;startTime=2019-02-12T02:43:01&amp;endTime=2019-02-12T04:14:38&amp;location=02&amp;channel=BH?&amp;nodata=404`

In [8]:
def download_mseed(event, channel='BH?', data_path=DATA_PATH + 'raw/'):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass

    payload = {
        'network': event['network'],
        'station': event['station'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'location': event['location'],
        'channel': channel
    }
    
    req = requests.get('http://ws.ipgp.fr/fdsnws/dataselect/1/query', params=payload)
    file_name = '-'.join([event['network'], event['station'], event['location'], event['startTime'], event['endTime']]) + '.mseed'
    print('downloading: %s' % file_name)
    path = os.path.join(data_path, file_name)
    with open(path, 'wb') as c:
        c.write(req.content)
    return path

In [9]:
# download all examples:
for event in ava:
    download_mseed(event)
    sleep(0.1)

downloading: XB-ELYSE-02-2019-02-12T02:43:01.313000-2019-02-12T04:14:37.259000.mseed
downloading: XB-ELYSE-02-2019-02-12T06:39:54.299000-2019-02-12T07:10:02.247000.mseed
downloading: XB-ELYSE-02-2019-02-15T08:14:08.907000-2019-02-15T08:42:48.854000.mseed
downloading: XB-ELYSE-02-2019-02-19T11:56:36.366000-2019-02-20T12:36:19.190000.mseed
downloading: XB-ELYSE-02-2019-02-25T06:48:00.718000-2019-02-25T06:52:21.668000.mseed
downloading: XB-ELYSE-02-2019-02-28T07:48:10.343000-2019-02-28T07:57:28.292000.mseed
downloading: XB-ELYSE-02-2019-02-28T11:59:55.327000-2019-03-01T04:04:14.200000.mseed
downloading: XB-ELYSE-02-2019-03-01T00:00:04.781000-2019-03-01T00:00:10.231000.mseed
downloading: XB-ELYSE-02-2019-03-01T08:17:33.222000-2019-03-01T14:18:31.170000.mseed
downloading: XB-ELYSE-02-2019-03-01T14:52:03.222000-2019-03-02T05:48:00.129000.mseed
downloading: XB-ELYSE-02-2019-03-02T06:19:20.175000-2019-03-03T05:24:54.997000.mseed
downloading: XB-ELYSE-02-2019-03-03T16:56:00.030000-2019-03-04T14

In [5]:
with open(os.path.join(DATA_PATH + 'raw/', 'catelog.json'), 'w') as f:
    json.dump(ava, f)

NameError: name 'ava' is not defined

In [6]:
file_names = list(filter(lambda f: os.path.splitext(f)[1] == '.mseed', os.listdir(DATA_PATH + 'raw/')))
print('num of files: %i' % len(file_names))

num of files: 145


In [7]:
streams = []
for file in file_names:
    streams.append(obspy.read(os.path.join(DATA_PATH + 'raw/', file)))

In [15]:
streams[4]

1 Trace(s) in Stream:
XB.ELYSE.02. | 2019-04-17T15:10:22.668000Z - 2019-04-17T15:16:58.618000Z | 20.0 Hz, 7920 samples

As some streams have more than 3 traces (in multiples of 3), we must go through each stream, cut each trace, and rejoin traces to their matching other 2 traces. This is accomplished in the following. 

This next box is a test for a stream with a small time frame in order to ensure the code works. 

In [19]:
windows = []
sliced = []
for trace in streams[0]:
    start = UTCDateTime(trace.stats.starttime)
    end = UTCDateTime(trace.stats.endtime)
    while start + 12 < end:
        sliced.append((trace.slice(start,start+12)))
        start = start+3
for aSlice in sliced:
    window = []
    window.append(aSlice)
    sliced.remove(aSlice)
    for bSlice in sliced:
        if bSlice.stats.starttime == aSlice.stats.starttime:
            window.append(bSlice)
            sliced.remove(bSlice)
    stream = obspy.core.Stream(window)
    stream.write('.data/prepared/{}.mseed'.format(aSlice.stats.starttime), format='MSEED')


KeyboardInterrupt: 

This following box is to run for all streams. Takes a VERY long time so be prepared!

In [ ]:
for stream in streams:
    print(stream)
    sliced = []
    for trace in stream:
        start = UTCDateTime(trace.stats.starttime)
        end = UTCDateTime(trace.stats.endtime)
        while start + 12 < end:
            sliced.append((trace.slice(start,start+12)))
            start = start+3
    print('saving {} traces'.format(traces))
    for aSlice in sliced:
        window = []
        window.append(aSlice)
        sliced.remove(aSlice)
        for bSlice in sliced:
            if bSlice.stats.starttime == aSlice.stats.starttime:
                window.append(bSlice)
                sliced.remove(bSlice)
        stream = obspy.core.Stream(window)
        file_name = '.data/prepared/{}.mseed'.format(aSlice.stats.starttime)
        print('saving {}'.format(file_name))
        stream.write(file_name, format='MSEED')

In [2]:
files = os.listdir('.data/prepared/')

In [ ]:
len(files)

In [ ]:
st = read(os.path.join('.data/prepared', files[5988668]))
st

5988669

In [8]:
st = read(os.path.join('.data/prepared', files[5988668]))
st

3 Trace(s) in Stream:
XB.ELYSE.02.BHU | 2019-10-14T04:44:50.680000Z - 2019-10-14T04:45:02.680000Z | 20.0 Hz, 241 samples
XB.ELYSE.02.BHV | 2019-10-14T04:44:50.680000Z - 2019-10-14T04:45:02.680000Z | 20.0 Hz, 241 samples
XB.ELYSE.02.BHW | 2019-10-14T04:44:50.680000Z - 2019-10-14T04:45:02.680000Z | 20.0 Hz, 241 samples

[    0.       137.9125  1165.825    806.7375  2213.65     727.5625
   617.475   1617.3875  1194.3     1918.2125  1177.125   1522.0375
  1594.95     469.8625  1904.775    736.6875   357.6     2241.5125
   799.425   1296.3375  1345.25    -171.8375   167.075    327.9875
   904.9      589.8125  1640.725   2336.6375   677.55    1569.4625
  1698.375    165.2875  1449.2     2786.1125   882.025    229.9375
  1046.85     604.7625  1680.675   1330.5875    19.5     1309.4125
  2018.325   1628.2375   785.15    1256.0625  1493.975     38.8875
   691.8     1234.7125  1126.625   1442.5375   416.45     522.3625
  1547.275   1344.1875   766.1       15.0125   828.925   2165.8375
   945.75     174.6625   853.575    750.4875   126.4     1116.3125
  1574.225    448.1375   992.05    1061.9625  1560.875   2338.7875
  1309.7      102.6125  -468.475   1630.4375  2652.35     700.2625
   663.175    493.0875  -907.       610.9125  2051.825   1578.7375
  1171.65     253.5625  1080.475   1802.3875   935.3      934.